<a href="https://colab.research.google.com/github/aysunakarsu/Simple/blob/master/Kaggle_oxford_102_flower_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c oxford-102-flower-pytorch

In [0]:
!unzip flower_data.zip

In [0]:
!mkdir ./flower_data/test/testdir

In [0]:
!cp ./flower_data/test/*.jpg ./flower_data/test/testdir/

In [0]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('Not your lucky day! Training on CPU ...')
else:
    print('Yay! Training on GPU ...')

In [0]:
!pip install --no-cache-dir -I pillow
# Imports here
%matplotlib inline
import time
import os
import json
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
from google.colab import files
import numpy as np

In [0]:
import PIL
print(PIL.PILLOW_VERSION)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
!pip install -U -q PyDrive
# !git clone https://github.com/Joshua1989/python_scientific_computing.git
!git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
!mv /tmp/colab_util_repo/colab_util.py colab_util.py 
!rm -r /tmp/colab_util_repo

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def load_model(filepath):
    checkpoint = torch.load(filepath)
    model = models.densenet161(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'], strict=False)
    model.eval() 
    
    return model

In [0]:
model = load_model('/content/drive/My Drive/densenet161_checkpoint.pth')

In [0]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [0]:
data_dir = './flower_data'
data_transforms = {
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
     'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
    
}
# TODO: Load the datasets with ImageFolder


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['valid']}

image_datasets['test'] =  ImageFolderWithPaths(os.path.join(data_dir, 'test'),
                                          data_transforms['test'])                                   
                                              
batch_size = 32

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['valid']}

dataloaders['test'] = torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=False, num_workers=0)


dataset_sizes = {x: len(image_datasets[x]) for x in ['valid','test']}

print("Datasets Load has finished:")
print("\tNumber of validation images:{}".format(dataset_sizes['valid']))
print("\tNumber of test images:{}".format(dataset_sizes['test']))
len(class_names)

In [0]:


def kaggle_prep(k_model,k_device, k_idx_to_class, k_dataloaders, phase):
    predictions = []
    image_names = []
    
    # Requires use of test_dataloader as it has paths component
    
    k_model.eval()
    k_model.to(device)   

    with torch.no_grad():
        for data, target, paths in k_dataloaders[phase]:
            
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, pred = torch.max(output, 1)
            for idx in range(len(target.data)):            
                image_names.append(paths[idx].split('/')[-1])
                predictions.append(idx_to_class[pred[idx].item()])
             
   
    return  np.vstack((filenames, predictions))

In [0]:
idx_to_class = {v: k for k, v in image_datasets['valid'].class_to_idx.items()}

kaggle_test_images = kaggle_prep(model, device,idx_to_class, dataloaders,'test')
print(idx_to_class)

In [0]:
import pandas as pd
df = pd.DataFrame({'file_name': kaggle_test_images[0], 'id': kaggle_test_images[1]})
print (df)
df.to_csv('submission_02.csv' ,index=False)
df.describe()